In [29]:
from dotenv import load_dotenv
import os 
from langchain_community.chat_models import ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS,DocArrayInMemorySearch
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [30]:
load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')
Model = 'gpt-3.5-turbo'


In [32]:
model = ChatOpenAI(api_key=API_KEY,model=Model)
model.invoke('what is the capita of UAS?')

AIMessage(content='The capital of the United States is Washington, D.C.', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 15, 'total_tokens': 27}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-737354f6-55b1-44e0-bbe9-af933c7344b2-0', usage_metadata={'input_tokens': 15, 'output_tokens': 12, 'total_tokens': 27})

In [33]:
parser = StrOutputParser()
chain = model|parser

In [34]:
chain.invoke('what is coding?')

'Coding is the process of using a programming language to create instructions that a computer can understand and execute. It involves writing lines of code to create software, websites, apps, or other types of digital content. Coding requires logical thinking, problem-solving skills, and attention to detail.'

In [35]:
file_loader = PyPDFLoader('malnutrition detection.pdf')
page = file_loader.load_and_split()
len(page)

7

In [36]:
spliter = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)
pages = spliter.split_documents(page)
pages[3]

Document(page_content='of Science and Technology,  \nChennai, India.  \nsaravanan_cse_2019@crescent.educati\nonPavani B  \nDepartment of Computer Science and \nEngineering,  \nB. S. Abdur Rahman Crescent Institute', metadata={'source': 'malnutrition detection.pdf', 'page': 0})

In [37]:
vector_storage = FAISS.from_documents(pages,OpenAIEmbeddings())
retriever = vector_storage.as_retriever()

In [39]:
question_template = """
your a smart bot that answers questions based on the context given to you only.
You don't make things up.
context:{context}
question:{question}

"""

In [40]:
prompt = PromptTemplate.from_template(template=question_template)
print(prompt.format(context = ' Here is the context to use',
              question = ' Answer this question based on the context'
              ))


your a smart bot that answers questions based on the context given to you only.
You don't make things up.
context: Here is the context to use
question: Answer this question based on the context




In [41]:
result = RunnableParallel(context= retriever,question = RunnablePassthrough())

In [42]:
chain = result |prompt | model | parser

In [43]:
chain.invoke('how many classes does the model classifies the input image into?')

'The model classifies the input image into 3 categories.'

In [44]:
retriever.invoke('how many classes does the model classifies the input image into?')

[Document(page_content='and it is observed that the model achieved 96% of accuracy \nfor the learning rate of 0.001 as shown in Fig 6.  The final \nresults were classified into 3 categories based upon image', metadata={'source': 'malnutrition detection.pdf', 'page': 3}),
 Document(page_content='one million images from the ImageNet database and can \nclassify images into 1000 object categories, such as a \nkeyboard, mouse, pencil, and many animals. As a result, the', metadata={'source': 'malnutrition detection.pdf', 'page': 2}),
 Document(page_content='classification and all the images in the dataset resized to the \ndimensions of 227x227x3 for training and testing process.  \n \nFig. 5.  Training progress of the network', metadata={'source': 'malnutrition detection.pdf', 'page': 2}),
 Document(page_content='images, and differentiates one from the other. The architecture \nwe used here is AlexNet for the training process and Transfer \nLearning. The system takes the image of a child as 